In [1]:
import os
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta

from google.cloud import bigquery
from google.oauth2 import service_account

pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('ga4_data.csv')

/var/folders/mk/p5y6cznx60788rys6_dnljxm0000gn/T/ipykernel_62745/4055328101.py:1: DtypeWarning: Columns (1,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ga4_data.csv')


In [3]:
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'])
df['totals_totalTransactionRevenue'] = df['totals_totalTransactionRevenue'].fillna(0)
#df['hits_eCommerceAction_action_type'] = df['hits_eCommerceAction_action_type'].fillna(0)
df['hits_transaction_transactionRevenue'] = df['hits_transaction_transactionRevenue'].fillna(0)
#df.loc[(df.trafficSource_medium == 'organic') & (df.trafficSource_keyword.isnull()), 'trafficSource_keyword'] = '(not provided)'
#df.loc[(df.trafficSource_medium.isin(["(none)", 'affiliate'])) & (df.trafficSource_keyword.isnull()), 'trafficSource_keyword'] = '(not set)'

df.visitStartTime = (df.visitStartTime - timedelta(hours=7))
df = df.assign(month = df.visitStartTime.dt.strftime('%Y-%m'))
df = df.assign(date = df.visitStartTime.dt.strftime('%m-%d'))
df = df.assign(day_name = df.visitStartTime.dt.weekday)

In [4]:
# pd.set_option('display.max_rows',None)     # 모든 행 보기
#pd.reset_option('display.max_rows')  

1. 소비금액의 경우 hits_transaction_transactionRevenue 컬럼 사용
2. visitStartTime - 7시간 
3. hits_hitNumber , hits_hour, hits_minute, hits_type, hits_isExit 컬럼 의미

- 제품 목록 클릭 = 1(event), (quickview click, product click )
- 제품 세부정보 보기 = 2(page),
- 장바구니에 제품 추가 = 3(event), add to cart
- 장바구니에서 제품 제거 = 4
- 결제 = 5(page),
- 구매 완료 = 6(page),
- 구매 환불 = 7,
- 결제 옵션 = 8, 
- 알 수 없음 = 0.

In [12]:
def 한글변환(col):
    if col == 0:
        return "알수없음(0)"
    elif col == 1:
        return "제품 목록 클릭(1)"
    elif col == 2:
        return "세부 정보 보기(2)"
    elif col == 3:
        return "장바구니 추가(3)"
    elif col == 4:
        return "장바구니 제거(4)"
    elif col == 5:
        return "결제(5)"
    elif col == 6:
        return "구매 완료(6)"
    elif col == 7:
        return "구매 환불(7)"
    else:
        return "결제 옵션(8)"
    
df["hits_eCommerceAction_action_type"] = df["hits_eCommerceAction_action_type"].apply(한글변환)

In [31]:
temp3 = df[df.fullVisitorId == 3095756132882918441]
temp3 = temp3[temp3.visitNumber == 5].sort_values(['hits_hitNumber'])
temp3


,channelGrouping,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,totals_sessionQualityDim,totals_newVisits,totals_totalTransactionRevenue,totals_bounces,trafficSource_referralPath,trafficSource_campaign,trafficSource_medium,trafficSource_keyword,trafficSource_adContent,trafficSource_isTrueDirect,trafficSource_adwordsClickInfo_page,trafficSource_adwordsClickInfo_slot,hits_hitNumber,hits_hour,hits_minute,hits_eventInfo_eventCategory,hits_eventInfo_eventAction,hits_eventInfo_eventLabel,hits_eventInfo_eventValue,hits_type,hits_referer,hits_isExit,hits_page_pageTitle,hits_page_searchKeyword,hits_transaction_transactionRevenue,hits_transaction_transactionTax,hits_transaction_transactionShipping,hits_eCommerceAction_action_type,hits_eCommerceAction_step,hits_social_socialNetwork,hits_social_hasSocialSourceReferral,month,date,day_name
2189627,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,1,10,23,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Home,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
1093867,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,2,10,23,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Foam Can and Bottle Cooler,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
2463064,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,3,10,23,Enhanced Ecommerce,Add to Cart,NaN,NaN,EVENT,NaN,NaN,Foam Can and Bottle Cooler,NaN,0.0,NaN,NaN,3,1,(not set),No,2016-08,08-30,1
546369,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,4,10,23,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Shopping Cart,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
546370,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,5,10,23,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Store search results,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
2463065,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,6,10,24,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Office,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
1093868,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,7,10,24,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Office,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
1916586,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,8,10,24,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Lifestyle,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
2189628,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,9,10,24,NaN,NaN,NaN,NaN,PAGE,NaN,NaN,Store search results,NaN,0.0,NaN,NaN,0,1,(not set),No,2016-08,08-30,1
272304,Referral,3095756132882918441,1472577781,5,2016-08-30 10:23:01,Chrome,NaN,NaN,500.8,NaN,/,(not set),(none),NaN,NaN,True,NaN,NaN,10,10,24,Enhanced Ecommerce,Quickview Click,Recycled Mouse Pad,NaN,EVENT,NaN,NaN,Store search results,NaN,0.0,NaN,NaN,1,1,(not set),No,2016-08,08-30,1


In [4]:
# 2번 세부 정보 --> 3번 장바구니의 퍼널
# 2번에서 3번으로 바로 넘어가는 경우 (전환된 세션), 2번에서 다른곳으로 넘어가는 경우 (전환되지 않은 세션) 비교
# 전환된 세션은 2번과 3번이 찍혀있는 세션.
# 전환되지 않은 세션은 2번은 찍혔는데 3번은 찍히지 않은 세션. 
# 전환된 세션에서 액션 컬럼을 시간 순서로 나열한다. 나열한뒤 카운트를 세본다.

# "2 and 3" 이 있는 세션만 추출
selected_visits = df.groupby('visitId').filter(lambda x: (2 in x['hits_eCommerceAction_action_type'].values) and (3 in x['hits_eCommerceAction_action_type'].values))

# 마지막으로 찍힌 2의 hitNumber 가 마지막으로 찍힌 3의 hitNumber 보다 작을때, 전환된 세션이라고 정의한다.
전환세션 = []
for visit_id, group in selected_visits.groupby('visitId'):
    group = group.sort_values(['hits_hitNumber'])
    ind1 = group[group['hits_eCommerceAction_action_type'] == 2].hits_hitNumber.iloc[-1]
    ind2 = group[group['hits_eCommerceAction_action_type'] == 3].hits_hitNumber.iloc[-1]
    if ind1 < ind2:
        전환세션.append(visit_id)

len(전환세션)

24797

In [ ]:
# # proof
# temp4 = selected_visits[selected_visits['visitId'].isin(전환세션)]
# for visit_id, group in temp4.groupby('visitId'):
#     group = group.sort_values(['hits_hitNumber'])
#     print(visit_id)
#     print(group['hits_eCommerceAction_action_type'])

In [5]:
# 전환되지 않은 세션은 2번이 기록된 세션중 전환세션이 아닌 세션이다.
selected_visits = df.groupby('visitId').filter(lambda x: 2 in x['hits_eCommerceAction_action_type'].values)
비전환세션 = selected_visits[~selected_visits['visitId'].isin(전환세션)]['visitId'].unique()
len_selected = selected_visits['visitId'].nunique()
print(f'{len_selected} = {len(전환세션)} + {len(비전환세션)}')

78921 = 24797 + 54124


In [8]:
df[df['visitId'].isin(전환세션)]['hits_transaction_transactionRevenue'].sum()

1209747.5299999998

In [9]:
df[df['visitId'].isin(비전환세션)]['hits_transaction_transactionRevenue'].sum()

140054.54

In [ ]:
# 세션 아이디로 그룹해서 unique() 로만 보면 더 편하고 효율적이다. 
# 한번이라도 등장하면 unique()에 등장하기 때문이다.
# UNNEST(hits) : hits를 컬럼별로 분리시키기
# _TABLE_SUFIX : table 이름이 ~로 끝나는 경우
# hits.hitNumber=1 : 랜딩페이지로 한정

query = f"""
SELECT
    hits.page.pagePath AS landing_page,
    COUNT(*) AS views,
    SUM(totals.bounces)/COUNT(*) AS bounce_rate
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'  AND
    hits.type = 'PAGE'  AND
    hits.hitNumber=1
GROUP BY landing_page
ORDER BY 2 DESC
LIMIT 10
"""
result_bounce = client.query(query).result().to_dataframe()
result_bounce['label'] = result_bounce['landing_page'].str.split('/').str[-1]
result_bounce